In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import gsw
import numpy as np
from scipy.integrate import solve_ivp
import cmocean.cm as cm

In [2]:
Iona_wNuts = xr.open_dataset('/ocean/cstang/results/WWTP/Iona/01jul21_30d_wNutrients/WWTP_1h_20210701_20210731_grid_T.nc')

mesh = xr.open_dataset('/home/sallen/MEOPAR/grid/mesh_mask202108.nc')
tmask = 1 - mesh.tmask[0]

WWInput = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/OAE/DyeTracing/Iona/wastewater_Iona_wNutrients.nc')

Iona_X = 305
Iona_Y = 447
outfall_depth = 95

mesh.mbathy[0,446,305].values,mesh.mbathy[0,447,305].values,mesh.mbathy[0,447,304].values,mesh.mbathy[0,446,305].values

(array(26, dtype=int16),
 array(27, dtype=int16),
 array(28, dtype=int16),
 array(26, dtype=int16))

In [3]:
DepthIdx = 28
TPlume = WWInput.temperature[6,Iona_Y,Iona_X].values
z_levels = mesh.gdepw_1d[0,0:28]

rho_ref = 1000
rho_fresh = gsw.rho(0,TPlume,0)
g = 9.81
alpha = 0.1

### Initial Conditions
Q0 = WWInput.flux[6,Iona_Y,Iona_X].values*500*440/1025*4
D_pipe = 5 #of pipe
R = D_pipe/2
Area = (D_pipe/2)**2
w0 = Q0/Area

## Ambient temperature
region_temp = Iona_wNuts.votemper[0,:,Iona_Y-3:Iona_Y+3,Iona_X-3:Iona_X+3]
Tamb = region_temp.where(region_temp > 0, drop=True).mean(dim=['x','y'])

## Ambient salinity
region_sal = Iona_wNuts.vosaline[0,:,Iona_Y-3:Iona_Y+3,Iona_X-3:Iona_X+3]
Samb = region_sal.where(region_sal > 0, drop=True).mean(dim=['x','y'])

## Ambient density
region_rho = Iona_wNuts.sigma_theta[0,:,Iona_Y-3:Iona_Y+3,Iona_X-3:Iona_X+3]
rho_amb = region_rho.where(region_rho > 0, drop=True).mean(dim=['x','y']) + 1000
gp_init = 9.81*(rho_amb[27].values - rho_fresh)/rho_ref

M0 = w0**2*Area
B0 = gp_init*w0*Area
M0 = Q0**2/(Area)

### Buoyancy Frequency 

drho_amb = np.zeros(28)
z_levels_idx = np.arange(28,0,-1)

for ii,z_idx in enumerate(z_levels_idx):
    drho_amb[z_idx-1] = (rho_amb[z_idx] - rho_amb[z_idx-1])/(mesh.gdept_1d[0,z_idx]-mesh.gdept_1d[0,z_idx-1])
    
N2 = -9.81/rho_ref * drho_amb

### Rise height in uniformly stratified fluid

From Equation 29. in https://sites.ualberta.ca/~bsuther/papers/plumespread/reprint.pdf 

#### Check constraints

$\frac{M_0 * N}{F_0}$ >>1 or <<1

In [4]:
N = np.sqrt(np.mean(abs(N2[26]))) # initial buoyancy flux

print('Momentum flux (m4/s2):', M0)
print('Buoyancy flux (m4/s3):',B0)
print('N (1/s):',N)

Momentum flux (m4/s2): 3.047015190365215
Buoyancy flux (m4/s3): 1.090250464180952
N (1/s): 0.007455893940647338


In [5]:
constraint_check = M0*N/B0
print('M0*N/F0:',constraint_check)

M0*N/F0: 0.020837617447814077


Check if $\frac{M_0}{F_0}$ < $\frac{7}{N}$

In [6]:
print(M0/B0)
print(7/N)

2.7947845843425325
938.8545566398232


Much smaller, buoyancy dominates

In [7]:
zs = 2.7*(B0/(N**3))**(1/4)
print('Rise height above source (m):',zs)

Rise height above source (m): 108.73529649493092


#### Check with mean buoyancy frequency of 40 m to 100 m

In [23]:
N = np.sqrt(np.mean(abs(N2[23:26]))) # mean buoyancy flux

print('Momentum flux (m4/s2):', M0)
print('Buoyancy flux (m4/s3):',B0)
print('N (1/s):',N)

Momentum flux (m4/s2): 3.047015190365215
Buoyancy flux (m4/s3): 1.090250464180952
N (1/s): 0.011136368053069078


In [24]:
constraint_check = M0*N/B0
print('M0*N/F0:',constraint_check)

M0*N/F0: 0.031123749760282118


In [25]:
print(M0/B0)
print(7/N)

2.7947845843425325
628.5711792787655


In [26]:
zs = 2.7*(B0/(N**3))**(1/4)
print('Rise height above source (m):',zs)

Rise height above source (m): 80.47994156577782
